In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from datetime import datetime
from datetime import timedelta
from dateutil import rrule
import time

import os
import glob

sns.set_theme(style="dark")
sns.color_palette("viridis", as_cmap=True)

plt.rcParams["font.family"] = "sans-serif"
plt.style.use('dark_background')

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.despine(left=True)

<Figure size 640x480 with 0 Axes>

### Equity Data Uploading

In [7]:
AMZN = pd.read_csv('Historical_Data_Prices/AMZN.csv')

#### Data Integrity Testing

In [10]:
AMZN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6738 entries, 0 to 6737
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          6738 non-null   object 
 1   Open          6738 non-null   float64
 2   High          6738 non-null   float64
 3   Low           6738 non-null   float64
 4   Close         6738 non-null   float64
 5   Volume        6738 non-null   int64  
 6   Dividends     6738 non-null   float64
 7   Stock Splits  6738 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 421.2+ KB


In [11]:
AMZN.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1997-05-15 00:00:00-04:00,0.121875,0.125000,0.096354,0.097917,1443120000,0.0,0.0
1,1997-05-16 00:00:00-04:00,0.098438,0.098958,0.085417,0.086458,294000000,0.0,0.0
2,1997-05-19 00:00:00-04:00,0.088021,0.088542,0.081250,0.085417,122136000,0.0,0.0
3,1997-05-20 00:00:00-04:00,0.086458,0.087500,0.081771,0.081771,109344000,0.0,0.0
4,1997-05-21 00:00:00-04:00,0.081771,0.082292,0.068750,0.071354,377064000,0.0,0.0
